In [1]:
from ovcg import *
from fesol import *
from pyccx import *
import numpy as np
import scipy.io as sio
from interface import DynamicUnderRelaxation, InterfaceData, RelativeCovergenceMonitor, AbsCovergenceMonitor
from mapper import generate_mapper

In [2]:
k=20
# problem discription:
# basically, with a fixed Pr=0.01 and Re=500, the k is the ratio between
# kappa_s/kappa_f, where kappa stands for conductivity coefficient.
assert k == 1 or k == 5 or k == 20

ccx_logoff = True
if ccx_logoff:
    import logging
    logger = logging.getLogger('PYCCX')
    logger.setLevel(logging.CRITICAL)

use_relax = True

# tolerance setting
tol = 1e-6
if tol > 1e-1:
    print('The tolerance is too large, this may lead to instability or low quality results')

In [3]:
# fluid solver
interface_grid = 3
interface_side = 0
interface_axis = 1

solverF = Cgins()
solverF.read_cmd('fluid_inputs/fluid_flat_plate%i_fnsd.cmd' % k)
solverF.init_grid()
solverF.init_flux_interface(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)
solverF.init_solver(debug=0, logname='fluid_outputs/dummy')
fluid_ofile = VtkFile('fluid_outputs/resFNSD%i.pvd' % k)
fnodes = solverF.get_coordinates_interface(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)

In [4]:
# solid solver
jobname = 'solid/solidD'
output = 'asc'
prob = Problem(jobname=jobname, output=output)
msolver = prob.get_solver()
msolver.initailize()
# since we know there is one step
# we stop b4 it
msolver.solve(stopb4step=1)
nlgeom = msolver.get_nlgeomsolver()
nlgeom.initialize()

# initialize our interface set
iset = nlgeom.get_set(
    set_name='Interface',
    set_type='sfelem',
    is_surface=True
)
nset = nlgeom.get_set(
    set_name='Inodes',
    set_type='node',
    is_surface=True
)

# retrieve geometry data
snodes = nset.coordinates()
scents = iset.extract_face_centers()

# create cht adapter
solverS = CHTAdapter(nlgeom=nlgeom)
solverS.add_interface(set_acc=nset, is_settable=True, itype=CHTAdapter.TEMP)
solverS.add_interface(set_acc=iset, is_settable=False, itype=CHTAdapter.DFLUX)

# now we are ready

In [5]:
Tf = 'Tf'
Ts = 'Ts'
Ff = 'Ff'
Fs = 'Fs'
Ff2Fs = {Tf:Ts}
Ts2Tf = {Fs:Ff}
f2s_mapper, s2f_mapper = generate_mapper(fnodes, [snodes,scents], Ff2Fs, Ts2Tf,r1=0.05,r2=0.2)

In [6]:
# coupling data and setups
fluxF = InterfaceData(size=fnodes.shape[0], value=0.0)
fluxS = InterfaceData(size=scents.shape[0], value=0.0)
tempF = InterfaceData(size=fnodes.shape[0], value=300.0)
tempS = InterfaceData(size=snodes.shape[0], value=310.0)
thetaT = InterfaceData(size=fnodes.shape[0], value=0.0)

under_relax = DynamicUnderRelaxation(init_omega=1.0)
conv_mntr = RelativeCovergenceMonitor(tol=tol)
conv_solution = AbsCovergenceMonitor(tol=1e-4)

# maximum pc steps allowed
max_pc_steps = 200

In [ ]:
# coupling timing information, use fixed time step for now
# Tfinal = 1.0
# N = 100
# dt = Tfinal/N
dt = 0.05
t = 0.0
Nmax = 500
avg_pc_iterations = 0.0
flog = open('FNSD%i.log' % k, mode='w')
flog.write('Fluid Neumann with solid Dirichlet setting, study case: %i\n' % k)
flog.close()
flog = open('FNSD%i.log' % k, mode='a')

In [ ]:
# begin to couple
for step in range(Nmax):
    t += dt
    
    pc_counts = 0
    
    # back up solutions
    solverF.backup_current_solutions()
    solverS.backup_state()
    
    # mapping
    s2f_mapper.put(Fs, fluxS.curr)
    s2f_mapper.apply(Fs, Ff)
    fluxF.curr[:] = s2f_mapper.get(Ff)
    
    solverF.put_heat_flux_interface(
        grid=interface_grid,
        side=interface_side,
        axis=interface_axis,
        data=fluxF.curr
    )
    
    while True:
        # back up previous interface value
        fluxF.backup()
        
        # advance fluid
        solverF.advance(t=t, dt=dt)
        # retrieve fluid interface temperature
        tempF.curr[:] = solverF.get_temperature_interface(
            grid=interface_grid,
            side=interface_side,
            axis=interface_axis
        )
        
        # mapping
        f2s_mapper.put(Tf, tempF.curr)
        f2s_mapper.apply(Tf, Ts)
        tempS.curr[:] = f2s_mapper.get(Ts)
        # update solid temperature
        solverS['Inodes', SET].set_temperatures(tempS.curr)
        
        # advance solid
        solverS.adjust_timesize(dt)
        solverS.increment()
        # retrieve solid interface heat flux
        fluxS.curr[:] = solverS['Interface', GET].get_dfluxes()
        
        # mapping
        s2f_mapper.put(Fs, fluxS.curr)
        s2f_mapper.apply(Fs, Ff)
        fluxF.curr[:] = s2f_mapper.get(Ff)
        
        # update residual
        fluxF.update_res()
        
        # test convergence
        is_conv = conv_mntr.determine_convergence(fluxF)
        if is_conv or pc_counts >= max_pc_steps:
            solverS.finish_increment()
            break
        else:
            # if not converge, then underrelaxation and update to fluid then restore
            if use_relax:
                under_relax.determine_omega(fluxF)
                under_relax.update_solution(fluxF)
            solverF.put_heat_flux_interface(
                grid=interface_grid,
                side=interface_side,
                axis=interface_axis,
                data=fluxF.curr
            )
            solverF.restore_previous_solutions()
            solverS.restore_state()
            solverS.finish_increment()
            pc_counts = pc_counts + 1

    # outputs
    msg = 'step=%i, pc_iterations=%i.' % (step, pc_counts)
    print(msg)
    flog.write(msg+'\n')
    avg_pc_iterations += pc_counts
    if step % 50 == 0 or step == Nmax-1:
        fluid_ofile.write(solverF, t)
flog.close()

nlgeom.finalize()
msolver.solve(skipsolve=True)
msolver.finalize()

step=0, pc_iterations=29.
step=1, pc_iterations=27.
step=2, pc_iterations=25.
step=3, pc_iterations=25.
step=4, pc_iterations=24.
step=5, pc_iterations=25.
step=6, pc_iterations=21.
step=7, pc_iterations=23.
step=8, pc_iterations=24.
step=9, pc_iterations=21.
step=10, pc_iterations=21.
step=11, pc_iterations=21.
step=12, pc_iterations=21.
step=13, pc_iterations=21.
step=14, pc_iterations=20.
step=15, pc_iterations=20.
step=16, pc_iterations=21.
step=17, pc_iterations=21.
step=18, pc_iterations=20.
step=19, pc_iterations=19.
step=20, pc_iterations=19.
step=21, pc_iterations=19.
step=22, pc_iterations=20.
step=23, pc_iterations=20.
step=24, pc_iterations=20.
step=25, pc_iterations=20.
step=26, pc_iterations=20.
step=27, pc_iterations=20.
step=28, pc_iterations=20.
step=29, pc_iterations=19.
step=30, pc_iterations=19.
step=31, pc_iterations=19.
step=32, pc_iterations=19.
step=33, pc_iterations=19.
step=34, pc_iterations=19.
step=35, pc_iterations=19.
step=36, pc_iterations=19.
step=37, pc

step=297, pc_iterations=13.
step=298, pc_iterations=13.
step=299, pc_iterations=14.
step=300, pc_iterations=13.
step=301, pc_iterations=12.
step=302, pc_iterations=13.
step=303, pc_iterations=12.
step=304, pc_iterations=13.
step=305, pc_iterations=13.
step=306, pc_iterations=13.
step=307, pc_iterations=13.
step=308, pc_iterations=12.
step=309, pc_iterations=13.
step=310, pc_iterations=12.
step=311, pc_iterations=12.
step=312, pc_iterations=13.
step=313, pc_iterations=12.
step=314, pc_iterations=12.
step=315, pc_iterations=12.
step=316, pc_iterations=12.
step=317, pc_iterations=13.
step=318, pc_iterations=12.
step=319, pc_iterations=12.
step=320, pc_iterations=12.
step=321, pc_iterations=12.
step=322, pc_iterations=12.
step=323, pc_iterations=12.
step=324, pc_iterations=12.
step=325, pc_iterations=12.
step=326, pc_iterations=12.
step=327, pc_iterations=12.
step=328, pc_iterations=12.
step=329, pc_iterations=12.
step=330, pc_iterations=12.
step=331, pc_iterations=12.
step=332, pc_iterati

In [ ]:
theta=(solverF.get_temperature_interface(3,0,1)[0:100]-300)/10
print(theta)
sio.savemat(
    'numeric_res/kFNSD%i.mat' % k,
    {
        'theta': theta,
        'avg_pc_its': avg_pc_iterations/(step+1),
        'N': (step+1)
    }
)